In [33]:
import pandas as pd

```{mermaid}{mermaid}
graph LR
Papermill -->|0-parameters|D
D<--> |1-Lectura datos| C[(Storage)]
D-->|2-Preprocesamiento|D
D--> |3-Guarda datos preprocesado|C
```

# Params

In [ ]:
INPUT_DATA = ""
OUPUT_DATA = ""

# Lectura de datos

In [34]:
df = pd.read_parquet(INPUT_DATA)
df.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,adult_male,alone
1,1,1,female,38.0,1,0,71.2833,C,False,False
3,1,1,female,35.0,1,0,53.1000,S,False,False
6,0,1,male,54.0,0,0,51.8625,S,True,True


# Encoder de variables categoricas

In [35]:
# Aplicar one-hot encoding a la variable embarked
embarked_onehot = pd.get_dummies(df['embarked'], prefix='embarked')
df.drop('embarked',axis=1, inplace=True)

# Concatenar el one-hot encoding al DataFrame original
df = pd.concat([df, embarked_onehot], axis=1)

In [ ]:
# Convertir datos categóricos a numéricos
df['sex'] = df['sex'].map({'female': 1, 'male': 0})

## Nuevas variables

In [36]:
# Crear variable "family_size" que representa el tamaño de la familia (sibsp + parch + 1)
df['family_size'] = df['sibsp'] + df['parch'] + 1

In [37]:
# Crear variable "is_child" que indica si la persona es menor de edad (edad menor a 18 años)
df['is_child'] = df['age'] < 18

In [38]:
# Crear variable "age_group" que indica en qué rango de edades se encuentra la persona
bins = [0, 18, 30, 50, 100]
labels = ['0-18', '19-30', '31-50', '51+']
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels)
age_group_onehot = pd.get_dummies(df['age_group'], prefix='age_group')#,drop_first=True
df = pd.concat([df, age_group_onehot], axis=1)
df.drop('age_group',axis=1,inplace=True)

## Guardar datos preprocesados

In [39]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,adult_male,alone,embarked_C,embarked_Q,embarked_S,family_size,is_child,age_group_0-18,age_group_19-30,age_group_31-50,age_group_51+
1,1,1,female,38.0,1,0,71.2833,False,False,1,0,0,2,False,0,0,1,0
3,1,1,female,35.0,1,0,53.1000,False,False,0,0,1,2,False,0,0,1,0
6,0,1,male,54.0,0,0,51.8625,True,True,0,0,1,1,False,0,0,0,1
10,1,3,female,4.0,1,1,16.7000,False,False,0,0,1,3,True,1,0,0,0
11,1,1,female,58.0,0,0,26.5500,False,True,0,0,1,1,False,0,0,0,1


In [40]:
df.to_parquet(OUTPUT_DATA)